<a href="https://colab.research.google.com/github/Somani-Harsh/data-science-python/blob/master/Chinese_news_article_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install stopwordsiso

     |████████████████████████████████| 81kB 5.9MB/s 


In [4]:
!pip install transformers

     |████████████████████████████████| 1.0MB 8.8MB/s 
     |████████████████████████████████| 3.0MB 29.5MB/s 
     |████████████████████████████████| 1.1MB 59.3MB/s 
     |████████████████████████████████| 890kB 54.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0955c1843133c8a34babb33f5be0549c7d12dcc6dbd14463d1d1ff32a19d4ba6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
import pandas as pd
import numpy as np
import re
from stopwordsiso import stopwords
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
import torch
import tqdm
from transformers import get_linear_schedule_with_warmup

In [2]:
torch.cuda.is_available()

True

In [3]:
df = pd.read_csv("drive/My Drive/interviews/Sirion labs/train_data.csv")
print(df.shape)
df.head()

(2500, 3)


,Unnamed: 0,text,category
0,1497,﻿ 日月 光华 - - Traffic _ Info 精华区 文章 阅读- - - ...,traffic
1,436,﻿ 日本 去年 海外 资产 和 负债额 均 创 ...,economic
2,736,﻿ 梁斌 黄胄 联袂 在 京 举办 画展新华社 ...,art
3,2673,﻿ 记者来信 ： 乱 降价 也 有害新华社 北京...,economic
4,2266,﻿ 巴解 愿同 叙利亚 和解新华社 开罗 ５ 月...,politics


In [4]:
df = df[~df["text"].isna()]
df.shape

(2498, 3)

In [5]:
def preprocess_text(text):
  text = re.sub("-", " ", text)  # Remove dashes
  text = " ".join(list(filter(lambda x: x not in stopwords("zh"), text.split())))    # Remove Stopwords
  return text


In [6]:
df["text"] = df["text"].apply(preprocess_text)
df["text"] = df["text"].apply(lambda x: "[CLS] " + x + " [SEP]")

In [7]:
df["text"].str.split().apply(len).describe(percentiles=(0.5, 0.75, 0.8, 0.9, 0.95, 0.99))

count    2498.000000
mean      326.978383
std       495.234674
min        19.000000
50%       168.000000
75%       322.750000
80%       395.000000
90%       702.600000
95%      1199.100000
99%      2502.870000
max      6248.000000
Name: text, dtype: float64

In [8]:
le = LabelEncoder()
le.fit(df["category"])
num_labels = len(le.classes_)

LabelEncoder()

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
 
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
 
model = BertForSequenceClassification.from_pretrained("bert-base-chinese", 
                                                      num_labels=num_labels, hidden_dropout_prob=0.3)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
X_train, X_test, cat_train, cat_test = train_test_split(df["text"].tolist(), df["category"].tolist(), 
                                                    test_size=0.2, stratify=df["category"].tolist(), random_state=40)

In [12]:
encoding = tokenizer(X_train, return_tensors='pt', padding=True, truncation=True, \
                     max_length=512)

train_input_ids = encoding["input_ids"]
train_attention_mask = encoding["attention_mask"]

Truncating sequence length to 512 by assuming that this sequence length would be enough to predict the category of the news article

In [13]:
encoding = tokenizer(X_test, return_tensors='pt', padding=True, truncation=True, \
                     max_length=512)

test_input_ids = encoding["input_ids"]
test_attention_mask = encoding["attention_mask"]

In [14]:
y_train = torch.tensor(le.transform(cat_train))
y_test = torch.tensor(le.transform(cat_test))

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
batch_size = 8
n_epochs = 2
N = len(train_input_ids)
N

1998

In [17]:
model = model.to(device)

In [18]:
optimizer = torch.optim.AdamW(model.parameters() , lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=125*n_epochs)

In [19]:
n_epochs = 3

In [20]:
train_data = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, y_train)
test_data = torch.utils.data.TensorDataset(test_input_ids, test_attention_mask, y_test)

In [21]:
train_sampler = torch.utils.data.RandomSampler(train_data)
train_dataloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [22]:
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [23]:
model.train()

for epoch_i in range(n_epochs):
    print("Epoch : ", epoch_i)


    for step, batch in enumerate(tqdm.notebook.tqdm(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, label_ids = batch
        
        optimizer.zero_grad()
        
        logits = model(input_ids, input_mask, labels=None)[0]

        loss = criterion(logits.view(-1, 10), label_ids.view(-1))
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        accuracy = sum(np.argmax(logits.detach().cpu().numpy(), axis=1) == label_ids.cpu().numpy()) / len(label_ids)

        if step % 30 == 29:
            print("Step ", step, " Loss: ", loss, " Acc:", accuracy)


Epoch :  0


Step  29  Loss:  tensor(2.0479, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 0.25
Step  59  Loss:  tensor(1.2265, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 0.5
Step  89  Loss:  tensor(1.1652, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 0.625
Step  119  Loss:  tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  149  Loss:  tensor(0.2888, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  179  Loss:  tensor(0.8439, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 0.75
Step  209  Loss:  tensor(0.2013, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  239  Loss:  tensor(0.2022, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0

Epoch :  1


Step  29  Loss:  tensor(0.2407, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  59  Loss:  tensor(0.2466, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  89  Loss:  tensor(0.1673, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  119  Loss:  tensor(0.1573, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  149  Loss:  tensor(0.4560, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 0.875
Step  179  Loss:  tensor(0.2703, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  209  Loss:  tensor(0.1322, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  239  Loss:  tensor(0.2166, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0

Epoch :  2


Step  29  Loss:  tensor(0.1656, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  59  Loss:  tensor(0.2169, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  89  Loss:  tensor(0.1887, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  119  Loss:  tensor(0.1220, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  149  Loss:  tensor(0.3282, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  179  Loss:  tensor(0.1365, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0
Step  209  Loss:  tensor(0.4046, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 0.875
Step  239  Loss:  tensor(0.2160, device='cuda:0', grad_fn=<NllLossBackward>)  Acc: 1.0



In [24]:
torch.save(model.state_dict(), "drive/My Drive/interviews/Sirion labs/model.pickle")

In [25]:
torch.cuda.empty_cache()

In [26]:
torch.cuda.memory_cached(), torch.cuda.memory_allocated()

/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


(1889533952, 1700531712)

In [27]:
model.load_state_dict(torch.load("drive/My Drive/interviews/Sirion labs/model.pickle"))

<All keys matched successfully>

In [28]:
# model = model.cpu()
model = model.to(device)
_  = model.eval()
batch_size = 4

In [29]:
true_labels = 0
for step, batch in enumerate(tqdm.notebook.tqdm(test_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch  

    test_pred = model(input_ids, input_mask)[0]
    true_labels += sum(np.argmax(test_pred.detach().cpu().numpy(), axis=1) == label_ids.cpu().numpy())

print("Validation Accuracy :" , true_labels/len(y_test))


Validation Accuracy : 0.966
